# Portfolio Optimization

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import cvxpy as cp

## Class definition

In [4]:
class Portfolio:
    def __init__(self, assets, period="10y", q=0.1):
        """
        Initialize the Portfolio class with assets and settings.
        Args:
            assets (list): List of asset tickers.
            period (str): Historical data period (default is "10y").
            q (float): Risk aversion parameter (default is 0.1).
        """
        self.assets = assets
        self.period = period
        self.q = q
        self.n = len(assets)
        self.sigma = None
        self.r = None
        self._calculate_statistics()

    def _calculate_statistics(self):
        """
        Calculate the covariance matrix (sigma) and expected returns (r).
        """
        R = []
        for stock in self.assets:
            data = yf.Ticker(stock)
            df = data.history(period=self.period)
            close = df['Close'].values
            returns = np.diff(close) / close[:-1]
            R.append(returns)

        R = np.array(R).T
        self.r = np.mean(R, axis=0)
        self.sigma = np.cov(R.T)

    def calculate_optimal_x(self):
        """
        Calculate the optimal portfolio weights (x).
        Returns:
            np.array: Optimized portfolio weights.
        """
        n = len(self.assets)
        e = np.ones(n)
        X = cp.Variable(n)
        obj = cp.Minimize(self.q * cp.quad_form(X, self.sigma) -  self.r @ X)
        constraints = [e @ X == 1, X >= 0]
        problem = cp.Problem(obj, constraints)
        result = problem.solve()
        return X.value
    
    def calculate_optimal_with_transaction_costs(self, x_init, f_buy, f_sell):
        """
        Calculate the optimal portfolio weights with transaction costs.
        Args:
            x_init (np.array): Initial portfolio weights.
            f_buy (float): Buying transaction cost.
            f_sell (float): Selling transaction cost.
        Returns:
            tuple: Optimized portfolio weights, buy adjustments, sell adjustments.
        """
        e = np.ones(self.n)
        u_buy = cp.Variable(self.n)
        u_sell = cp.Variable(self.n)
        x = x_init + u_buy - u_sell

        objective = cp.Minimize(self.q * cp.quad_form(x, self.sigma) - self.r @ x)
        constraints = [
            x == x_init + u_buy - u_sell,
            x >= 0,
            u_buy >= 0,
            u_sell >= 0,
            (1 - f_sell) * e @ u_sell == (1 + f_buy) * e @ u_buy,
            e @ x == 1
        ]

        problem = cp.Problem(objective, constraints)
        result = problem.solve()

        return x.value, u_buy.value, u_sell.value


## Model with Transaction Costs

\begin{align}
    \min_{\mathbf{x}, \mathbf{u}_{\text{buy}}, \mathbf{u}_{\text{sell}}} & \quad q\cdot \mathbf{x}^T \Sigma \mathbf{x} + \mathbf{r}^T \mathbf{x}\\
    \text{subject to:} & \\
    & \mathbf{x} = \mathbf{x}_{\text{init}} + \mathbf{u}_{\text{buy}} - \mathbf{u}_{\text{sell}} \\
    & \mathbf{u}_{\text{buy}} \ge 0, \quad \mathbf{u}_{\text{sell}} \ge 0 \\
    & (1 - f_{\text{sell}}) \mathbf{1}^T \mathbf{u}_{\text{sell}} = (1 + f_{\text{buy}}) \mathbf{1}^T \mathbf{u}_{\text{buy}} \\
    & \mathbf{1}^T \mathbf{x} = 1
\end{align}

Methods already written before.

### Analysis

In [7]:
assets = ["TSLA", "NVDA", "AAPL", "JPM", "JNJ", "KO", "DUK", "T", "GE", "PFE"]  
portfolio = Portfolio(assets,"10y",10)
opt_x = portfolio.calculate_optimal_x()
x_init = np.full(10, 0.1)
opt_x_trans, *_ = portfolio.calculate_optimal_with_transaction_costs(x_init,0.01,0.01)
print(f"Optimal x with no transaction costs: {opt_x}\n"
      f"Initial x: {x_init}\n"
      f"Optimal x with transaction costs: {opt_x_trans}\n")

Optimal x with no transaction costs: [ 3.97508629e-02  1.30858245e-01  8.78184873e-03  1.48597298e-02
  2.89185903e-01  2.49739586e-01  1.54388290e-01  1.12435534e-01
 -2.65563953e-23  3.23498724e-23]
Initial x: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
Optimal x with transaction costs: [0.09999931 0.1        0.1        0.1        0.10000068 0.1
 0.1        0.1        0.1        0.1       ]



## Model allowing short positions

\begin{align*}
    \text{Minimize} \quad & q\cdot \mathbf{x}^T \Sigma \mathbf{x} + \mathbf{r}^T\mathbf{x} \\
    \text{subject to} \quad \\
    & \mathbf{x} = \mathbf{x}_{\text{long}} - \mathbf{x}_{\text{short}}, \\
    & \mathbf{x}_{\text{long}} \succeq 0, \quad \mathbf{x}_{\text{short}} \succeq 0, \\
    & \mathbf{1}^T \mathbf{x}_{\text{short}} \leq \eta \mathbf{1}^T \mathbf{x}_{\text{long}}, \\
    & \mathbf{1}^T \mathbf{x} = 1
\end{align*}

In [8]:
class PortfolioWithShort(Portfolio):
    def calculate_optimal_with_short(self, eta=0.5):
        """
        Calculate the optimal portfolio weights allowing short positions.
        Args:
            eta (float): Maximum ratio of short positions to long positions (default is 0.5).
        Returns:
            tuple: Optimized portfolio weights, long positions, short positions.
        """
        x_long = cp.Variable(self.n)
        x_short = cp.Variable(self.n)
        x = x_long - x_short

        objective = cp.Minimize(self.q * cp.quad_form(x, self.sigma) - self.r @ x)
        constraints = [
            x_long >= 0,
            x_short >= 0,
            cp.sum(x_short) <= eta * cp.sum(x_long),
            cp.sum(x) == 1
        ]

        problem = cp.Problem(objective, constraints)
        result = problem.solve()

        return x.value, x_long.value, x_short.value

In [9]:
assets = ["TSLA", "NVDA", "AAPL", "JPM", "JNJ", "KO", "DUK", "T", "GE", "PFE"]  
port_short = PortfolioWithShort(assets,"10y",10)
opt_x = portfolio.calculate_optimal_x()
opt_x_combined, opt_x_long, opt_x_short = port_short.calculate_optimal_with_short(0.5)
print(f"Optimal x with no transaction costs: {opt_x}\n"
      f"Optimal x: {opt_x_combined}\n"
      f"Optimal x long position: {opt_x_long}\n"
      f"Optimal x short position: {opt_x_short}\n")

Optimal x with no transaction costs: [ 3.97508629e-02  1.30858245e-01  8.78184873e-03  1.48597298e-02
  2.89185903e-01  2.49739586e-01  1.54388290e-01  1.12435534e-01
 -2.65563953e-23  3.23498724e-23]
Optimal x: [ 0.04100298  0.1350774   0.01172797  0.04240519  0.29340654  0.25915556
  0.15828841  0.12226905 -0.04995465 -0.01337843]
Optimal x long position: [0.07077539 0.18731385 0.05810994 0.06193282 0.39712931 0.35509729
 0.18317983 0.13513216 0.02984375 0.03733765]
Optimal x short position: [0.02977241 0.05223646 0.04638198 0.01952764 0.10372277 0.09594173
 0.02489142 0.01286311 0.0797984  0.05071608]

